### Basic Response
Express provides a `send` method which can respond with variety of response types like JSON, HTML, etc. It also automatically sets the `Content-Length` header as well. For example:

In [ ]:
res.send({ msg: 'Success' }) // Express sets content-type with application/json
// alternatively res.json({ msg: 'Success' })

res.send('Welcome') // Express sets content-type with text/html

We often combine `send` with `status` to set an HTTP response status

In [ ]:
res.status('500').send('An error occurred')

### File Download
To send a file for download (rather than display) the response should have the following parameters:
```
Content-Type: text/html; charset=utf-8
Content-Disposition: attachment; filename="some_song.mp3"
Content-Length: 21
```

The `download` method allows to send a response meant to be downloaded. It sets the above headers accrodingly

In [ ]:
app.get('/download', function (req, res) {
    res.download(
        'uploads/profile.jpg', // absolute path
        'profile.jpg' // optional file name
    )
})

### Response Header
To set response headers,

In [ ]:
res.set('Content-Type', 'text/plain')

res.set({
  'Content-Type': 'text/plain',
  'Content-Length': '125'
})

A number of methods are available which specify a specific header, for example:
- res.type : sets content-type
- res.cookie: sets cookies

### Cookies
An HTTP Cookie has a number of optional properties associated with it mostly related to cookie security. The list is:
- **max-age** number of seconds until the cookie expires
- **expires** date on which cookie expires
- **domain** what domain the cookie is available on, [more info](https://stackoverflow.com/questions/1062963/how-do-browser-cookie-domains-work)
- **secure** transfer cookie only over HTTPS
- **httponly** prevent browser JS from accessing cookie
- **path** the path that must exist in the requested URL, or the browser won't send the cookie header
- **samesite**

If the max-age or expires attribute is not present, then the cookie lasts only as long as the browser window (or tab) is open. Express also allows to create signed cookies which are encrypted using secret key (supplied to the cookie parser) middleware.

In [ ]:
const cookieParser = require('cookie-parser')
app.use(cookieParser('xH34dsjfaZ8*{'))

app.get('/', function (req, res) {
    // cookie name, value, options
    // setting single cookie
    res.cookie('language', 'en', {
        signed: true,
        maxAge: 3600000,
        httpOnly: true
    }).send('Welcome')
})

![Cookie header obtained by browser](https://i.imgur.com/Q8DDagn.png)  

Note that cookie keys are not encrypted (only values are).  
We can even pass an object as cookie value. It will get stringified.

In [ ]:
res.cookie('prefs', {
    language: 'en',
    region: 'APAC'
})

To remove cookie from client:

In [ ]:
res.clearCookie('language') // accepts options as second param

### Sessions
Since HTTP protocol is stateless, sessions are used to store user data. Session data often contains user identity and can either be stored client side or server side.  
**Client side session** session state is stored in client mostly as cookie (but can also be stored in LocalStorage). Advantages:
- no need to replicate session across nodes
- obtaining identity data from session is fast (just decrypt cookie data)
- no storage required on server  
Disadvantage:
- cookie has size limitation (non-issue most of the times)
- logout not fully possible (we can send a clearCookie command, but the user can just reattach the same cookie with every request)

![Client side session](https://i.imgur.com/Pw2oizu.png)

**Server side cookie** only a non private data stored as cookie on client side. The cookie data maps to actual user identification data and is stored in database or in memory.  
Advantages:
- full control of session, user session can be terminated when required
- smaller cookie size
Disadvantages:
- more points of failure (what if the database crashes?)
- more overhead while creating sessions  

![Server side session](https://i.imgur.com/BTfXptA.png)

`express-session` middleware is used for server side sessions. `cookie-parser` is not required, but the secret must be the same.

In [ ]:
const session = require('express-session')
let MemoryStore = require('memorystore')(session)

let sessionConfig = {
    secret: 'kjh^52JaLL.',  // required to sign cookies
    cookie: { // settings related to the session cookie
        secure: false,
        maxAge: null,
        httpOnly: true
    },
    store: new MemoryStore({ // where to store session? Lots of other options like Redis
        checkPeriod: 86400000 // how often should expired sessions be removed (24h here)?
    })
}

app.use(session(sessionConfig))

app.get('/', function(req, res){
    if(req.session.views){
        req.session.views++
    } else {
        req.session.views = 1
    }
    
    res.send(`Viewed ${req.session.views} times`)
})

### Templates
Express integrates readily with template engines like Pug and EJS. Below is the example of Pug being used as the template engine:

```pug
// index.pug
html
    head
        title= title
    body
        h1= greetings
```

In [ ]:
app.set('view engine', 'pug')
app.get('/', function(req, res){
    res.render('index', {
        title: 'My Web Page',
        greetings: 'Welcome Home!'
    })
})

### Redirecting

In [ ]:
res.redirect('/foo/bar')
res.redirect('http://example.com')
res.redirect(301, 'http://example.com')
res.redirect('../login') // one up and then redirect